In [5]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 10:17:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
spark.stop()

In [6]:
import os
directory = os.path.join(os.getcwd(),"data")
filename = "restaurant_reviews.csv"
filepath = os.path.join(directory, filename)
lines = spark.textFile("file:///"+filepath.replace("\\","/"))
lines

file:////home/lab12/src/data/restaurant_reviews.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [7]:
lines.take(5)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,']

In [12]:
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [12]:
header = lines.first()
filtered_lines = lines.filter(lambda row : row != header)

In [13]:
filtered_lines.take(5)

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,']

In [14]:
def parse(row):
    fields = row.split(",")
    category = fields[2]
    reviews = int(fields[3])
    
    return category, reviews

In [15]:
parse('0,짜장면,중식,125,')

('중식', 125)

In [ ]:
#RDD 내 모든 row 에 대해 parse()를 적용한 다음,map() 추출

In [16]:
category_reviews = filtered_lines.map(parse)


In [17]:
category_reviews.take(5)

[('중식', 125), ('중식', 235), ('분식', 32), ('분식', 534), ('일식', 223)]

In [18]:
category_review_count = category_reviews.mapValues(lambda x: (x,1))
category_review_count.take(3)

[('중식', (125, 1)), ('중식', (235, 1)), ('분식', (32, 1))]

In [20]:
reduced_rdd = category_review_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
reduced_rdd.take(3)
#카테고리별 합계

[('중식', (360, 2)), ('분식', (566, 2)), ('일식', (287, 3))]

In [22]:
reduced_rdd.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [23]:
#평균
average = reduced_rdd.mapValues(lambda x: x[0]/x[1])

In [24]:
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [ ]:
#데이터 추가 > 메뉴별 합계와 평균

In [1]:
spark.stop()

NameError: name 'spark' is not defined